This code was created following the tutorial: [hugging face - fine tune mistral](https://huggingface.co/blog/sirluk/multilabel-llm)

In [2]:
!pip install scikit-multilearn
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 970.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-many

In [52]:
exit()

In [2]:
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [3]:
import wandb
wandb.login()

# let's log every trained
# %env WANDB_LOG_MODEL=true

wandb_project_name = "LLP2-test"

wandb: Currently logged in as: rstern. Use `wandb login --relogin` to force relogin


In [4]:

from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [5]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  from google.colab import drive
  import sys
  drive.mount('/content/drive')
  sys.path.append('/content/drive/MyDrive/ucph/LP Project') # If working in collab change this path
  path = '/content/drive/MyDrive/ucph/LP Project/'

train_df = pd.read_csv(f'{path}df_train.csv')
val_df = pd.read_csv(f'{path}df_validation.csv')

# shuffle dataset
train_df = train_df.sample(frac=1, random_state=42)
val_df = val_df.sample(frac=1, random_state=42)
print(train_df.sample(5))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       Unnamed: 0                                         paragraph1  \
38110       38110  Besides the other responses, something you're ...   
18232       18232  But despite McConnell’s supposed disapproval o...   
20866       20866  Except it’s not a binary. There were plenty of...   
33567       33567  There's a difference between begrudgingly acce...   
10539       10539  Exactly. Republicans treat politics like it’s ...   

                                              paragraph2  label_author  \
38110  Everyone has their own pet issue they think sh...             0   
18232  In on April 7, 2022 with Axios reporter Jonath...             0   
20866  Video of a wagner POW who wanted to switch sid...             1   
33567  That would depend on if the Belarus military c...             1   
10539  I am not saying you did. I am saying if someth...            

In [6]:
print(train_df.columns)

def create_sequences(row):
  sequence = str(row["paragraph1"]) + "[LP2]" + str(row["paragraph2"])
  return sequence

train_df["input"] = train_df.apply(create_sequences, axis=1)
val_df["input"] = val_df.apply(create_sequences, axis=1)

x_train = train_df["input"].values
x_val = val_df["input"].values

def create_lable(row):
  label_dataset = max(min(1, row["label_dataset"]), 0)
  multilabel = np.array([int(row["label_author"]), int(label_dataset)])
  return multilabel

train_df["label"] = train_df.apply(create_lable, axis=1)
val_df["label"] = val_df.apply(create_lable, axis=1)

y_train = train_df["label"].values
y_train = np.stack(y_train)
y_val = val_df["label"].values
y_val = np.stack(y_val)

# label_weights = 1 - y_val.sum(axis=0) / y_val.sum()
label_weights = [2,1]


Index(['Unnamed: 0', 'paragraph1', 'paragraph2', 'label_author',
       'label_dataset', 'fileindex'],
      dtype='object')


In [7]:
print(x_train.shape, y_train.shape, y_train[0])
print(x_train[0])
print(x_val.shape, y_val.shape, y_val[0])
print(x_val[0])

(51993,) (51993, 2) [1 1]
That’s -80.86 ° F. I live in Montana and so far this winter it’s gotten down to -42°F. I can’t even imagine twice as cold as that and most people can’t even imagine -42°F.[LP2]Always amazes me that people can live in such extreme cold. Like as of this post, it’s -51 F (-41 C) real temp, but “feels like” -67 F (-55 C).
(11198,) (11198, 2) [0 1]
In general, be courteous to others. Debate/discuss/argue the merits of ideas, don't attack people. Personal insults, shill or troll accusations, hate speech, any suggestion or support of harm, violence, or death, and other rule violations can result in a permanent ban.[LP2]For those who have questions regarding any media outlets being posted on this subreddit, please click to review our details as to our approved domains list and outlet criteria.


In [8]:
x_train_sub = x_train[0:50]
y_train_sub = y_train[0:50]
x_val_sub = x_val[0:50]
y_val_sub = y_val[0:50]

debugg = True

In [9]:
if debugg:
  ds = DatasetDict({
    'train': Dataset.from_dict({'text': x_train_sub, 'labels': y_train_sub}),
    'val': Dataset.from_dict({'text': x_val_sub, 'labels': y_val_sub})
  })
else:
  ds = DatasetDict({
      'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
      'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
  })


Load the model

In [10]:
# model name
model_name = 'mistralai/Mistral-7B-v0.1'

# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

# load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=y_train.shape[1]
)

# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)

#####
# new tokens
new_tokens = ["[LP2]"]
# check if the tokens are already in the vocabulary
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
# add the tokens to the tokenizer vocabulary
tokenizer.add_tokens(list(new_tokens))
# add new, random embeddings for the new tokens
model.resize_token_embeddings(len(tokenizer))

tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')


#### FREEZE
"""
modules = [L1bb.embeddings, *L1bb.encoder.layer[:5]] #Replace 5 by what you want
for module in mdoules:
    for param in module.parameters():
        param.requires_grad = False

### check what was frozen
for name, param in model.named_parameters():
     print(name, param.requires_grad)
"""

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [11]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }


    # create custom trainer class to be able to pass label weights and calculate mutilabel loss
class CustomTrainer(Trainer):

    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss


In [12]:
wandb.init(entity="rstern", project="lp2" name="lp2-test")


In [13]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 13639680 || all params: 3634651136 || trainable%: 0.3752679277772644


In [14]:
# define training args
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 10,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    logging_steps=25,              # When to start reporting loss
    logging_dir="./logs",        # Directory for storing logs
    save_steps=25,                # Save checkpoints every 50 steps
    report_to = ["wandb"],
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['val'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    label_weights = torch.tensor(label_weights, device=model.device)
)

trainer.train()

# save model
peft_model_id = 'multilabel_mistral'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted
1,No log,2.108270,0.794872,0.781486,0.782687
2,No log,2.001128,0.794872,0.784759,0.785758
3,No log,1.476409,0.819876,0.819264,0.819442
4,1.231700,2.878096,0.710145,0.675926,0.677918
5,1.231700,1.727089,0.784314,0.773516,0.774617
6,1.231700,1.383490,0.839506,0.838127,0.838410
7,1.231700,1.800398,0.768212,0.753309,0.754666
8,0.044500,2.062856,0.765101,0.747693,0.749120
9,0.044500,1.927777,0.768212,0.753309,0.754666
10,0.044500,1.887045,0.776316,0.763566,0.764792


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=

('multilabel_mistral/tokenizer_config.json',
 'multilabel_mistral/special_tokens_map.json',
 'multilabel_mistral/tokenizer.model',
 'multilabel_mistral/added_tokens.json',
 'multilabel_mistral/tokenizer.json')

In [ ]:
# load model
# peft_model_id = 'multilabel_mistral'
# model = AutoModelForSequenceClassification.from_pretrained(peft_model_id)

In [49]:
wandb.finish()